In [ ]:
#Import lIBRARIES
import pandas as pd
import numpy as np
import os as os

In [ ]:
# example making new folder with os

os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
##Load Basics set and assign to DF

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

basicsDF = pd.read_csv(basics_url, sep='\t', low_memory=False)

basicsDF.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
#Assign local file to DF
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

In [ ]:
#Basics preprocessing

#Replace "\N" with np.nan
basics.replace({'\\N': np.nan}, inplace=True) 
basics.head()

In [ ]:
#Eliminate movies that are null for runtimeMinutes
basics = basics.dropna(subset=['genres', 'runtimeMinutes'])

In [ ]:
basics.isna().sum()

In [ ]:
#keep only titleType==Movie
movie_filter = basics['titleType']=='movie'
movie_filter.head()

basics[movie_filter].head()

In [ ]:
#keep startYear 2000-2022
start_filt = basics['startYear'] >= '2000'
start_filt.head()

In [ ]:
end_filt = basics['startYear']<='2022'
end_filt.head()

In [ ]:
basics = basics.loc[start_filt & end_filt, :]
basics.head()

In [ ]:
#Eliminate movies that include  "Documentary" in genre (see tip below)
#basics = basics[basics["genres"].str.contains("Documentary") == False]
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics

In [ ]:
pd.set_option("display.max_rows", None)
basics['genres'].value_counts()

In [ ]:
##Load AKAs data set and save locally 
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

akasDF = pd.read_csv(akas_url, sep='\t', low_memory=False)

akasDF.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [ ]:
#Assign local file to DF
akas = pd.read_csv("Data/title.akas.csv.gz", low_memory=False)
akas.head()

In [ ]:
akas.replace({'\\N':np.nan}, inplace=True)

In [ ]:
#AKA - keep only US entries. I keep breaking the main df, I'm creating a dummy
#df that I will move back once done.
dfa= akas
dfa.head()

In [ ]:
#Apply filter to retain only movies in US region
region_filt = dfa['region']=='US'
region_filt.head()

In [ ]:
#Check that filtering worked...it did.  ONLY US region films are showing.
dfa[region_filt].head()

In [ ]:
dfa = dfa.loc[region_filt, :]
dfa.head()

In [ ]:
dfa['region'].value_counts()

In [ ]:
#Transfer dummy set back to actual df
akas = dfa
akas.head()

In [ ]:
#Save new cleansed df to local csv file from earlier 
akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [ ]:
#Display df info as requested
akas.info()

In [ ]:
##Load Ratings data set and assign to DF
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

ratingsDF = pd.read_csv(ratings_url, sep='\t', low_memory=False)

ratingsDF.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [ ]:
#Assign local file to DF
ratings = pd.read_csv("Data/title.ratings.csv.gz", low_memory=False)
ratings.head()

In [ ]:
#RATINGS Preprocessing
ratings.replace({'\\N':np.nan}) 

In [ ]:
ratings.info()

In [ ]:
ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

In [ ]:
basics = basics[keepers]
basics

In [ ]:
basics = pd.to_csv("Data/title_basics.csv.gz", low_memory=False, compression= 'gzip',index=False)
basics.head()